In [32]:
from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
import numpy as np
import csv

overlay = Overlay("bitstreams/mlpv3_1.bit")


In [33]:
from pynq import allocate
from pynq import Overlay

WEIGHTS_01 = 30
WEIGHTS_12 = 150
WEIGHTS_23 = 75
WEIGHTS_34 = 40
WEIGHTS_45 = 20
WEIGHTS_56 = 3

dma = overlay.axi_dma_0

f = open("mlpv2.csv", "r")

weights = f.read().split(',')

for i in range(len(weights)):
    weights[i] = float(weights[i])

input_x = [0.0] * WEIGHTS_01

# input_list = weights + input_x
input_list = [0.0] * 19928
print(input_list)

print(len(input_list))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [3]:
input_buffer = allocate(shape=(19928,), dtype=np.float32)
for i in range(19928):
    input_buffer[i] = input_list[i]
    
res = allocate(shape=(3,), dtype=np.float32)

In [ ]:
import time

start_time = time.time()

dma.sendchannel.transfer(input_buffer)
dma.recvchannel.transfer(res)
dma.sendchannel.wait()
dma.recvchannel.wait()

print("-----%s ms elapsed for transfer-----" %(1000*(time.time()-start_time)))

print(res)

In [7]:
from pynq import Overlay
from pynq import allocate
import pynq.lib.dma
import numpy as np
from scipy.stats import iqr
from scipy.stats import skew

INPUT_SIZE = 30
OUTPUT_SIZE = 3

class Model:
    def __init__(self, bitfile, paramfile):
        self.overlay = Overlay(bitfile)
        self.dma = self.overlay.axi_dma_0
        
        f = open(paramfile, "r")
        self.params = f.read().split(',')
        for i in range(len(self.params)):
            self.params[i] = float(self.params[i])
        self.numofparams = len(self.params)
        
        self.input_buffer = allocate(shape=(self.numofparams+INPUT_SIZE,), dtype=np.float32)
        self.res = allocate(shape=(OUTPUT_SIZE,), dtype=np.float32)
    
    # raw_data is supposed to be 20*6
    def preprocess(self, raw_data):
        extracted_data = []
        for i in range(raw_data.shape[1]):
            extracted_data.append(np.mean(raw_data[:,i]))
        for i in range(raw_data.shape[1]):
            extracted_data.append(np.std(raw_data[:,i]))
        for i in range(raw_data.shape[1]):
            extracted_data.append(iqr(raw_data[:,i]))
        for i in range(raw_data.shape[1]):
            extracted_data.append(skew(raw_data[:,i]))
        for i in range(raw_data.shape[1]):
            extracted_data.append(min(raw_data[:,i]))
        for i in range(raw_data.shape[1]):
            extracted_data.append(max(raw_data[:,i]))
           
        return extracted_data    
        
        
    def classify(self, input_x):
        for i in range(OUTPUT_SIZE):
            self.input_buffer[self.numofparams+i] = input_x[i]
        self.dma.sendchannel.transfer(self.input_buffer)
        self.dma.recvchannel.transfer(self.res)
        self.dma.sendchannel.wait()
        self.dma.recvchannel.wait()
        return int(self.res[0])
        

In [15]:
# Sample of using the Model class above
mlp = Model("design_1_1.bit", "out.csv")
# fake input here, shld be a list with 12 floats
input_x = [12.0] * 12
# classify function will return a class index (integer) with highest probability
mlp.classify(input_x)


0

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt 
import h5py as h5
motion_data_directory = "Logs for week 11"

from os import listdir
from os.path import isfile, join
file_path = './' + motion_data_directory
all_files = [f for f in listdir(file_path) if isfile(join(file_path, f))]

def obtain_side_and_action_df(side, dance):
  df = pd.read_csv('./trial2/' + dance + '/' + side + '.csv')
  temp = []
  for i in df.columns:
    temp.append(side + '_' + i)
  df.columns = temp
  return df

def join_df(left, right):
  return pd.concat([left, right], axis=1)

def get_joined_df(move):
  temp_df_list = []
  for file in all_files:
    if move in file:
      temp_df = pd.read_csv(file_path + '/' + file)
      temp_df_list.append(temp_df)
  
  df = pd.concat(temp_df_list)
  return df

side_pump = get_joined_df('Sidepump')
hair = get_joined_df('Hair')
gun = get_joined_df('Gun')

movement_list = [side_pump, hair, gun]
# movement_list = [downstairs_df_list]
action_reference = [1,2,3]
movement = ["side_pump", "hair", "gun"]

df_combined = pd.DataFrame(columns=[ 
       'accel.x', 'accel.y',
       'accel.z', 'gyro.x', 'gyro.y', 'gyro.z',
       'action'])

for i in range(len(movement_list)):
  action = movement_list[i]
  action['action'] = movement[i]
  frames = [df_combined, action]
  df_combined = pd.concat(frames)

df_temp = pd.DataFrame()
df_temp["action"] = df_combined['action']
df_combined = df_combined.drop(['action'], axis=1)
df_combined = df_combined.drop(['timestamp'], axis=1)
df_combined['action'] = df_temp['action']
df_combined

,accel.x,accel.y,accel.z,gyro.x,gyro.y,gyro.z,action
0,0.103884,-0.074545,0.376018,0.007634,-0.015267,0.000000,side_pump
1,0.116458,-0.097298,0.379012,0.015267,0.000000,-0.015267,side_pump
2,0.084125,-0.069755,0.407453,0.015267,0.000000,0.000000,side_pump
3,0.104183,-0.054187,0.363444,-0.007634,0.015267,-0.007634,side_pump
4,0.091310,-0.115260,0.336500,0.068702,0.015267,0.038168,side_pump
5,0.068857,-0.114961,0.337698,-0.007634,0.068702,-0.053435,side_pump
6,0.119452,-0.114362,0.390987,0.038168,-0.015267,-0.030534,side_pump
7,0.174537,-0.220342,0.192200,0.083969,-0.038168,-0.030534,side_pump
8,0.170645,-0.159269,0.209564,-0.053435,0.038168,-0.030534,side_pump
9,0.119452,-0.125439,0.326321,-0.030534,0.015267,0.015267,side_pump


In [35]:
df_combined = df_combined.reset_index()
df_combined = df_combined.drop(columns=['index'])
df_combined
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()
df_combined['action_label'] = label.fit_transform(df_combined['action'])
label.classes_

array(['gun', 'hair', 'side_pump'], dtype=object)

In [36]:
col_names = []
for col_name in df_combined.columns:
  col_names.append(col_name)

X = df_combined[col_names[0:6]]
y = df_combined['action_label']

from sklearn.preprocessing import  StandardScaler

scaler = StandardScaler()

X = scaler.fit_transform(X)
scaled_X = pd.DataFrame(data = X, columns = col_names[0:6])
scaled_X['action_label'] = y.values

In [37]:
import scipy.stats as stats

data_frequency = 10 #50hz
frame_size = data_frequency * 6
sliding = data_frequency * 3

def get_frames(df, frame_size, sliding):
  num_features = 6

  frames = []
  labels = []
  for i in range(0, len(df) - frame_size, sliding):
    accel_x = df['accel.x'].values[i:i+frame_size]
    accel_y = df['accel.y'].values[i:i+frame_size]
    accel_z = df['accel.z'].values[i:i+frame_size]
    gyro_x = df['gyro.x'].values[i:i+frame_size]
    gyro_y = df['gyro.y'].values[i:i+frame_size]
    gyro_z = df['gyro.z'].values[i:i+frame_size]

    label = stats.mode(df['action_label'][i:i+frame_size])[0][0]
    frames.append([accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z])
    labels.append(label)
  
  frames = np.asarray(frames).reshape(-1, frame_size, num_features)
  labels = np.asarray(labels)
  return frames, labels

In [38]:
X, y = get_frames(scaled_X, frame_size, sliding)
X.shape

(417, 60, 6)

In [45]:
from scipy.stats import iqr
from scipy.stats import skew

# Extracted features:
# Mean acceleration & rotation
mean_uAx = []
mean_uAy = []
mean_uAz = []
mean_uRx = []
mean_uRy = []
mean_uRz = []

# S.d. acceleration & rotation
sd_uAx = []
sd_uAy = []
sd_uAz = []
sd_uRx = []
sd_uRy = []
sd_uRz = []

# iqr acceleration & rotation
iqr_uAx = []
iqr_uAy = []
iqr_uAz = []
iqr_uRx = []
iqr_uRy = []
iqr_uRz = []

# skewness
skew_uAx = []
skew_uAy = []
skew_uAz = []
skew_uRx = []
skew_uRy = []
skew_uRz = []

# minimum
min_uAx = []
min_uAy = []
min_uAz = []
min_uRx = []
min_uRy = []
min_uRz = []

# Maximum
max_uAx = []
max_uAy = []
max_uAz = []
max_uRx = []
max_uRy = []
max_uRz = []

#Energy
energy_uAx = []
energy_uAy = []
energy_uAz = []
energy_uRx = []
energy_uRy = []
energy_uRz = []

def calculate_energy(accel):
  delta_time = 2 #2 second window
  velocity = integrate.quad(lambda x: accel, 0, delta_time)
  energy = 0.5 * 75 * (velocity**2)
  return energy

for i in range(len(X)):
  mean_uAx.append(np.mean(X[i], axis=0)[0])
  mean_uAy.append(np.mean(X[i], axis=0)[1])
  mean_uAz.append(np.mean(X[i], axis=0)[2])
  mean_uRx.append(np.mean(X[i], axis=0)[3])
  mean_uRy.append(np.mean(X[i], axis=0)[4])
  mean_uRz.append(np.mean(X[i], axis=0)[5])

  sd_uAx.append(np.std(X[i], axis=0)[0])
  sd_uAy.append(np.std(X[i], axis=0)[1])
  sd_uAz.append(np.std(X[i], axis=0)[2])
  sd_uRx.append(np.std(X[i], axis=0)[3])
  sd_uRy.append(np.std(X[i], axis=0)[4])
  sd_uRz.append(np.std(X[i], axis=0)[5])
  
  iqr_uAx.append(iqr(X[i], axis=0)[0])
  iqr_uAy.append(iqr(X[i], axis=0)[1])
  iqr_uAz.append(iqr(X[i], axis=0)[2])
  iqr_uRx.append(iqr(X[i], axis=0)[3])
  iqr_uRy.append(iqr(X[i], axis=0)[4])
  iqr_uRz.append(iqr(X[i], axis=0)[5])

  skew_uAx.append(skew(X[i], axis=0)[0])
  skew_uAy.append(skew(X[i], axis=0)[1])
  skew_uAz.append(skew(X[i], axis=0)[2])
  skew_uRx.append(skew(X[i], axis=0)[3])
  skew_uRy.append(skew(X[i], axis=0)[4])
  skew_uRz.append(skew(X[i], axis=0)[5])

  min_uAx.append(X[i].min(axis=0)[0])
  min_uAy.append(X[i].min(axis=0)[1])
  min_uAz.append(X[i].min(axis=0)[2])
  min_uRx.append(X[i].min(axis=0)[3])
  min_uRy.append(X[i].min(axis=0)[4])
  min_uRz.append(X[i].min(axis=0)[5])

  max_uAx.append(X[i].max(axis=0)[0])
  max_uAy.append(X[i].max(axis=0)[1])
  max_uAz.append(X[i].max(axis=0)[2])
  max_uRx.append(X[i].max(axis=0)[3])
  max_uRy.append(X[i].max(axis=0)[4])
  max_uRz.append(X[i].max(axis=0)[5])

# Create 2D Matrix for Decision Tree
feature_extracted_data = pd.DataFrame(
    {'mean_uAx': mean_uAx,
     'mean_uAy': mean_uAy,
     'mean_uAz': mean_uAz,
     'mean_uRx': mean_uRx,
     'mean_uRy': mean_uRy,
     'mean_uRz': mean_uRz,
     'sd_uAx': sd_uAx,
     'sd_uAy': sd_uAy,
     'sd_uAz': sd_uAz,
     'sd_uRx': sd_uRx,
     'sd_uRy': sd_uRy,
     'sd_uRz': sd_uRz,
     'iqr_uAx': iqr_uAx,
     'iqr_uAy': iqr_uAy,
     'iqr_uAz': iqr_uAz,
     'iqr_uRx': iqr_uRx,
     'iqr_uRy': iqr_uRy,
     'iqr_uRz': iqr_uRz,
    #  'skew_uAx': skew_uAx,
    #  'skew_uAy': skew_uAy,
    #  'skew_uAz': skew_uAz,
    #  'skew_uRx': skew_uRx,
    #  'skew_uRy': skew_uRy,
    #  'skew_uRz': skew_uRz,
     'min_uAx': min_uAx,
     'min_uAy': min_uAy,
     'min_uAz': min_uAz,
     'min_uRx': min_uRx,
     'min_uRy': min_uRy,
     'min_uRz': min_uRz,
     'max_uAx': max_uAx,
     'max_uAy': max_uAy,
     'max_uAz': max_uAz,
     'max_uRx': max_uRx,
     'max_uRy': max_uRy,
     'max_uRz': max_uRz,
    })

mat = feature_extracted_data.copy()
reorder = pd.DataFrame()
col_list = ['mean_uAx',
     'mean_uAy',
     'mean_uAz',
     'mean_uRx',
     'mean_uRy',
     'mean_uRz',
     'sd_uAx',
     'sd_uAy',
     'sd_uAz',
     'sd_uRx',
     'sd_uRy',
     'sd_uRz',
     'iqr_uAx',
     'iqr_uAy',
     'iqr_uAz',
     'iqr_uRx',
     'iqr_uRy',
     'iqr_uRz',
     'min_uAx',
     'min_uAy',
     'min_uAz',
     'min_uRx',
     'min_uRy',
     'min_uRz',
     'max_uAx',
     'max_uAy',
     'max_uAz',
     'max_uRx',
     'max_uRy',
     'max_uRz']
for col_name in col_list:
    reorder[col_name] = mat[col_name]
mat = reorder.copy()
mat.values[1]

# import pandas as pd
# import numpy as np

# mat = pd.read_csv('matrix.csv')
# mat = mat.drop(['Unnamed: 0'], axis=1)
# mat.head()

array([-0.01688014,  0.20425504,  0.25986374, -0.01438675,  0.11033859,
        0.21483339,  0.81401207,  0.7868395 ,  0.84553577,  0.88501655,
        0.73491086,  0.91642739,  0.75911006,  1.02194744,  0.9819761 ,
        0.76325804,  0.82803771,  0.92682509, -2.03475409, -1.45261567,
       -2.99509309, -2.90612619, -1.20148287, -2.38457868,  1.75842943,
        2.47733425,  2.22407922,  1.56921615,  2.16797716,  2.49207257])

In [40]:
# y = np.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
#        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
#        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
#        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
#        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
#        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
#        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(mat, y, test_size=0.9, random_state = 0, stratify = y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape
type(X_test.values[1][2])

numpy.float64

In [42]:
from pynq import Overlay
from pynq import allocate
import pynq.lib.dma
import numpy as np
import pandas as pd
from pynq import Xlnk


# Setup overlay
overlay = Overlay("bitstreams/mlpv2_1.bit")
dma = overlay.axi_dma_0

WEIGHTS_01 = 30
WEIGHTS_12 = 150
WEIGHTS_23 = 75
WEIGHTS_34 = 40
WEIGHTS_45 = 20
WEIGHTS_56 = 3

f = open("mlpv2.csv", "r")

weights = f.read().split(',')

for i in range(len(weights)):
    weights[i] = np.float32(weights[i])


input_buffer = allocate(shape=(19928,), dtype='f4')
input_buffer1 = allocate(shape=(100,), dtype='f4')
input_buffer2 = allocate(shape=(28,), dtype='f4')
res = allocate(shape=(4,), dtype='f4')

In [43]:
correct = 0

for k in range(X_test.shape[0]):
    input_x = list(X_test.values[k])
    for j in range(30):
        input_x[j] = float(input_x[j])
    input_list = weights + input_x

#     for i in range(199):
#         for j in range(100):
#             index = i*100+j
#             input_buffer1[j] = np.float32(input_list[index])
#         dma.sendchannel.transfer(input_buffer1)
#     #     dma.sendchannel.wait()

#     for i in range(28):
#         input_buffer2[i] = np.float32(input_list[i+19900])
#     dma.sendchannel.transfer(input_buffer2)
    
    for i in range(19928):
        input_buffer[i] = np.float32(input_list[i])
    
    dma.sendchannel.transfer(input_buffer)


    dma.recvchannel.transfer(res)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

    # print(input_buffer)
    prediction = np.argmax(res)
    print(prediction, y_test[k])    
    print(res)
    if (prediction == y_test[k]):
        correct += 1

    
print("TOTAL ACCURACY: ", float(correct/X_test.shape[0]))
    
    

0 0
[9.8528743e-01 7.1605458e-04 1.3996451e-02 0.0000000e+00]
2 2
[0.20251332 0.22919844 0.5682882  0.        ]
2 0
[0.0452855  0.03367383 0.9210407  0.        ]
2 1
[0.1994611  0.22673936 0.57379955 0.        ]
2 2
[0.08605681 0.4269475  0.48699564 0.        ]
0 2
[9.4998670e-01 5.7490448e-05 4.9955752e-02 0.0000000e+00]
2 2
[0.27090707 0.0963347  0.6327582  0.        ]
0 0
[7.3571980e-01 2.4391308e-04 2.6403627e-01 0.0000000e+00]
2 2
[0.02390048 0.04715216 0.9289473  0.        ]
1 1
[0.00649627 0.9428281  0.05067562 0.        ]
2 0
[0.09427088 0.03682102 0.86890805 0.        ]
1 2
[0.16760296 0.5478666  0.28453043 0.        ]
0 0
[0.99602306 0.00103432 0.00294259 0.        ]
1 2
[0.3070664  0.44572535 0.24720821 0.        ]
2 1
[0.21030731 0.32300964 0.46668306 0.        ]
0 0
[0.7859123  0.0346809  0.17940684 0.        ]
1 2
[0.10281277 0.6068606  0.29032665 0.        ]
2 0
[0.10135747 0.00107112 0.89757144 0.        ]
0 0
[9.9843460e-01 5.0334254e-04 1.0620328e-03 0.0000000e+00]
1 

1 1
[7.9517916e-04 8.0558747e-01 1.9361734e-01 0.0000000e+00]
2 2
[1.8651278e-01 3.3928256e-04 8.1314796e-01 0.0000000e+00]
1 0
[0.00508863 0.9823829  0.01252842 0.        ]
0 0
[9.9848396e-01 6.9626214e-05 1.4464204e-03 0.0000000e+00]
2 0
[0.07639845 0.0222364  0.90136516 0.        ]
0 0
[0.99144065 0.0024706  0.00608875 0.        ]
0 2
[8.8634980e-01 3.8470133e-04 1.1326555e-01 0.0000000e+00]
1 1
[0.0108858  0.6139389  0.37517533 0.        ]
1 1
[0.00565078 0.8579668  0.13638243 0.        ]
0 0
[0.98963493 0.00527486 0.0050902  0.        ]
0 2
[9.9480057e-01 1.3117710e-04 5.0681685e-03 0.0000000e+00]
0 0
[0.99198043 0.00213636 0.00588323 0.        ]
0 2
[0.6280924  0.00214235 0.36976522 0.        ]
2 2
[0.03187174 0.11995997 0.84816825 0.        ]
2 2
[0.07537917 0.02933915 0.8952817  0.        ]
2 1
[0.11686388 0.28714818 0.595988   0.        ]
0 0
[0.9934068  0.0018264  0.00476683 0.        ]
2 1
[0.32051614 0.10514526 0.5743386  0.        ]
2 2
[0.23755579 0.15205844 0.6103858  0.

1 1
[0.09363395 0.47484547 0.4315206  0.        ]
2 2
[0.10621656 0.0731516  0.82063186 0.        ]
2 2
[0.0152068  0.09293898 0.89185417 0.        ]
0 0
[0.8193901  0.14401875 0.03659117 0.        ]
2 0
[0.15717076 0.29060596 0.5522233  0.        ]
1 1
[0.04905357 0.8736717  0.07727475 0.        ]
2 2
[0.06256046 0.00903703 0.92840254 0.        ]
0 0
[0.977882   0.01997887 0.00213904 0.        ]
1 1
[0.00937009 0.86229724 0.12833269 0.        ]
2 2
[0.08529372 0.32984832 0.58485794 0.        ]
0 0
[0.88491243 0.00093604 0.11415145 0.        ]
2 0
[0.3030105  0.01408048 0.6829091  0.        ]
0 0
[9.9744081e-01 7.8862696e-04 1.7705782e-03 0.0000000e+00]
1 1
[0.0054956  0.80295265 0.19155172 0.        ]
2 0
[0.16115893 0.03140877 0.80743235 0.        ]
0 0
[0.79079217 0.00609621 0.20311159 0.        ]
1 1
[4.9925601e-04 6.9835889e-01 3.0114192e-01 0.0000000e+00]
1 1
[0.00174047 0.96377265 0.03448683 0.        ]
2 0
[0.4464749 0.0014733 0.5520517 0.       ]
2 0
[0.12287509 0.2267766  0.6

In [ ]:
import time

start_time = time.time()

dma.sendchannel.transfer(input_buffer)
dma.recvchannel.transfer(res)
dma.sendchannel.wait()
dma.recvchannel.wait()

print("-----%s ms elapsed for transfer-----" %(1000*(time.time()-start_time)))

print(res)